In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta8"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta8"
#r "nuget: Microsoft.ML, 1.5.5"
#r "nuget: Microsoft.ML.Mkl.Components, 1.5.5"
#r "nuget: Microsoft.ML.TimeSeries, 1.5.5"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha5"

Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json

Installed package Plotly.NET.Interactive version 2.0.0-beta8

Installed package Microsoft.ML version 1.5.5

Installed package Microsoft.ML.TimeSeries version 1.5.5

Installed package Plotly.NET version 2.0.0-beta8

...

Installed package FSharp.Stats version 0.4.1

Loading extensions from `Deedle.DotNet.Interactive.Extension.dll`

Added DeedleFormatterExtension including formatters for Frame and Series

Loading extensions from `Plotly.NET.Interactive.dll`

Added Kernel Extension including formatters for GenericChart

In [ ]:
open Deedle
open Plotly.NET

In [ ]:
let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

,Ticks,Value
,System.Int32,System.Int32
01.01.2015 01:00:00,1,5946
01.01.2015 02:00:00,2,5726
01.01.2015 03:00:00,3,5347
01.01.2015 04:00:00,4,5249
01.01.2015 05:00:00,5,5309
01.01.2015 06:00:00,6,5574
01.01.2015 07:00:00,7,5925
01.01.2015 08:00:00,8,6343
01.01.2015 09:00:00,9,6882


In [ ]:
let shiftedValues =
    data?Value
    |> Series.shift -1

let baseDateSeries =
    Seq.zip data.RowKeys data.RowKeys
    |> Series.ofObservations

let dayOfWeek =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.DayOfWeek)

let month =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.Month)

let peakOffPeak =
    baseDateSeries
    |> Series.mapValues (fun dt -> if dt.Hour < 8 || dt.Hour > 19 then "OffPeak" else "Peak")

let dataWithFeatures =
    data
    |> Frame.addCol "Target" shiftedValues
    |> Frame.addCol "DayOfWeek" dayOfWeek
    |> Frame.addCol "Month" month
    |> Frame.addCol "PeakOffPeak" peakOffPeak
    |> Frame.filterRows (fun key _ -> key.Year < 2020)
    |> Frame.dropSparseRows

let dataTrain =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year < 2019)

let dataTest =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year >= 2019)

dataTrain
|> Frame.skip 3

,Ticks,Value,Target,DayOfWeek,Month,PeakOffPeak
,System.Int32,System.Int32,System.Double,System.String,System.String,System.String
01.01.2015 04:00:00,4,5249,5309,Thursday,1,OffPeak
01.01.2015 05:00:00,5,5309,5574,Thursday,1,OffPeak
01.01.2015 06:00:00,6,5574,5925,Thursday,1,OffPeak
01.01.2015 07:00:00,7,5925,6343,Thursday,1,OffPeak
01.01.2015 08:00:00,8,6343,6882,Thursday,1,Peak
01.01.2015 09:00:00,9,6882,6963,Thursday,1,Peak
01.01.2015 10:00:00,10,6963,7110,Thursday,1,Peak
01.01.2015 11:00:00,11,7110,7136,Thursday,1,Peak
01.01.2015 12:00:00,12,7136,7013,Thursday,1,Peak


In [ ]:
dataTrain |> Frame.takeLast 1

,Ticks,Value,Target,DayOfWeek,Month,PeakOffPeak
,System.Int32,System.Int32,System.Double,System.String,System.String,System.String
31.12.2018 23:00:00,35063,6290,6075,Monday,12,OffPeak


In [ ]:
let oneDate = DateTime.Parse("2018-12-31T23:00:00")
let anotherDate = DateTime.Parse("2017-12-31T01:00:00")

(DateTimeOffset(oneDate) - DateTimeOffset(anotherDate)).TotalHours

8782

In [ ]:
type ILoadRow =
    abstract member Ticks: float32 with get
    abstract member Value: float32 with get
    abstract member Target: float32 with get
    abstract member DayOfWeek: string with get
    abstract member Month: string with get
    abstract member PeakOffPeak: string with get

let (trainKeys: DateTime seq, trainRows: ILoadRow seq) =
    dataTrain.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

let testKeys, testRows =
    dataTest.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

Seq.length trainRows, Seq.length testRows

Item1,Item2
35063,8760


In [ ]:
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Trainers
open Microsoft.ML.Transforms
open FSharp.Stats.Correlation

In [ ]:
[<CLIMutable>]
type ForecastInput =
    { Ticks: float32
      Value: float32
      [<ColumnName("Label")>]Target: float32
      DayOfWeek: string
      Month: string
      PeakOffPeak: string }

    static member FromILoadRows (row: ILoadRow) =
        { Ticks = row.Ticks
          Value = row.Value
          Target = row.Target
          DayOfWeek = row.DayOfWeek
          Month = row.Month
          PeakOffPeak = row.PeakOffPeak }

[<CLIMutable>]
type ForecastResult =
    { [<ColumnName("Score")>]LoadForecast: float32 }

let downCastPipeline (pipeline: IEstimator<'a>) =
    match pipeline with
    | :? IEstimator<ITransformer> as p -> p
    | _ -> failwith $"The pipeline has to be an instance of IEstimator<ITransformer> but was %A{pipeline.GetType()}"

let mlContext = MLContext(seed = 42)
let defInp = Unchecked.defaultof<ForecastInput>
let dayOneHot = "DayOfWeekOneHot"
let monthOneHot = "MonthOneHot"
let peakOneHot = "PeakOffPeakOneHot"

let processingPipeline =
    EstimatorChain()
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(dayOneHot, nameof defInp.DayOfWeek))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(monthOneHot, nameof defInp.Month))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(peakOneHot, nameof defInp.PeakOffPeak))
        .Append(mlContext.Transforms.Concatenate("Features", [| dayOneHot; monthOneHot; peakOneHot; nameof defInp.Ticks |]))
        // .Append(mlContext.Transforms.Concatenate("Features", [| nameof defInp.Value |]))
        // .Append(mlContext.Transforms.Concatenate("Features", [| dayOneHot; monthOneHot; peakOneHot; nameof defInp.Ticks; nameof defInp.Value |]))
    |> downCastPipeline

let trainerOptions = OlsTrainer.Options(CalculateStatistics = true)
let trainer =
    mlContext.Regression.Trainers.Ols(trainerOptions)
    |> downCastPipeline

let dataViewTrain = mlContext.Data.LoadFromEnumerable<ForecastInput>(trainRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))
let dataViewTest = mlContext.Data.LoadFromEnumerable<ForecastInput>(testRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))

In [ ]:
let trainingPipeline = processingPipeline.Append(trainer)
let trainedModel = trainingPipeline.Fit(dataViewTrain)

In [ ]:
let transformedData = trainedModel.Transform(dataViewTest)
let predictions = mlContext.Data.CreateEnumerable<ForecastResult>(transformedData, reuseRowObject = false) |> Seq.toList

mlContext.Regression.Evaluate(transformedData)

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
697.2013361403931,783069.908636695,884.9123734227559,783069.9094340907,0.5916168662979051


In [ ]:
open Plotly.NET

let predVals = predictions |> Seq.map (fun p -> p.LoadForecast)
let actualVals = testRows |> Seq.map (fun r -> r.Target)

let predChart =
    Seq.zip testKeys predVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip testKeys actualVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

<!-- Plotly chart will be drawn inside this DIV -->

PValues are unfortunately unreliable. See [this issue](https://github.com/dotnet/machinelearning/issues/5696) for more context.

In [ ]:
let model = (trainedModel.LastTransformer :?> RegressionPredictionTransformer<Microsoft.ML.Trainers.OlsModelParameters>).Model
model

HasStatistics,StandardErrors,TValues,PValues,Weights,Bias,RSquared,RSquaredAdjusted
True,"[ 3842046.391187039, 2494388.277491061, 2494388.277491062, 2494388.2774910615, 2494388.277491062, 2494388.277491062, 2494388.2774910624, 2494388.277491062, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708, 1878492.1143290708 ... (3 more) ]","[ 0.001765230140018449, 0.00016330765120473213, 9.96789897190159E-05, -0.00026459347152027316, -0.00045825971244133084, 0.00010074692351391274, 0.00016657505786393334, 0.00019242295583452889, 0.00042496995002107974, 0.0005244814423021982, 0.0002727710984748647, -0.00013634638008364685, -0.0003758834370890424, -0.0002760717795197729, -0.00032326425945410336, -0.00037882062845135604, -0.00022828014429623203, -6.606732826488007E-05, 0.00026417958142728687, 0.00029799825040831296 ... (3 more) ]","[ 0.9985915422439575, 0.999869704246521, 0.9999204277992249, 0.9997888803482056, 0.9996343851089478, 0.999919593334198, 0.9998670816421509, 0.9998464584350586, 0.9996609091758728, 0.999581515789032, 0.999782383441925, 0.9998912215232849, 0.9997000694274902, 0.9997797012329102, 0.9997420907020569, 0.999697744846344, 0.9998178482055664, 0.9999473094940186, 0.9997892379760742, 0.999762237071991 ... (3 more) ]","[ 407.3527, 248.6381, -659.99884, -1143.0776, 251.30194, 415.50287, 479.97757, 798.3027, 985.23425, 512.3984, -256.1256, -706.09406, -518.59863, -607.2494, -711.6116, -428.82245, -124.10696, 496.25925, 559.78735, -721.894 ... (2 more) ]",6782.096,0.6370277015225914,0.6367998079561957


In [ ]:
let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

<!-- Plotly chart will be drawn inside this DIV -->

0.8025671590541936

In [ ]:
let predEngine = mlContext.Model.CreatePredictionEngine<ForecastInput, ForecastResult>(trainedModel)

let inputs =
    mlContext.Data.CreateEnumerable<ForecastInput>(dataViewTest, reuseRowObject = false)

let firstInput =
    Seq.head inputs
    |> fun fi -> { fi with Target = float32 0. }

let theRest =
    inputs
    |> Seq.skip 1
    |> Seq.map (fun fi -> { fi with Value = float32 0.; Target = float32 0. })
    |> Seq.toList

firstInput::theRest
|> List.take 5

index,Ticks,Value,Target,DayOfWeek,Month,PeakOffPeak
0,35064,6075,0,Tuesday,1,OffPeak
1,35065,0,0,Tuesday,1,OffPeak
2,35066,0,0,Tuesday,1,OffPeak
3,35067,0,0,Tuesday,1,OffPeak
4,35068,0,0,Tuesday,1,OffPeak


In [ ]:
let rec predictDynamically (toPredict: ForecastInput list) (lastPredcition: ForecastResult) =
    match toPredict with
    | [] -> []
    | x::xs ->
        let x_hat = { x with Value = lastPredcition.LoadForecast }
        let forecasted = predEngine.Predict(x_hat)
        forecasted::(predictDynamically xs forecasted)

let dynamicPredictions = predictDynamically (firstInput::theRest) { LoadForecast = firstInput.Value }
dynamicPredictions
|> List.take 10

index,LoadForecast
0,7933.752
1,7933.7705
2,7933.7896
3,7933.8086
4,7933.827
5,7933.8457
6,7933.8647
7,7933.884
8,9376.774
9,9376.793


In [ ]:
let dynamicPredVals = dynamicPredictions |> List.map (fun pred -> pred.LoadForecast)

let predChart =
    Seq.zip testKeys dynamicPredVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip testKeys actualVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let modelDirectory = "../models"
let linearModel = modelDirectory + "/linear_model.zip"

mlContext.Model.Save(trainedModel, dataViewTrain.Schema, linearModel)

In [ ]:
[<CLIMutable>]
type AlternativeForecastInput =
    { Load: float32
      TimeStamp: DateTime }

[<CLIMutable>]
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

let altForecastInputs =
  dataTrain?Value
  |> Series.observations
  |> Seq.map (fun (k, v) -> { Load = float32 v; TimeStamp = k})

let pipeline =
  mlContext.Forecasting.ForecastBySsa(
    "Forecast",
    nameof Unchecked.defaultof<AlternativeForecastInput>.Load,
    windowSize =  24 * 30,
    seriesLength = 24 * 30 * 2,
    trainSize = dataTrain.RowCount,
    horizon = 24 * 30 ,
    confidenceLevel = 0.90f,
    confidenceLowerBoundColumn = "LowerBound",
    confidenceUpperBoundColumn = "UpperBound"
)

let altForecastData = mlContext.Data.LoadFromEnumerable(altForecastInputs)

let model = pipeline.Fit(altForecastData)

In [ ]:
open Microsoft.ML.Transforms.TimeSeries

let forecastingEngine = model.CreateTimeSeriesEngine<AlternativeForecastInput, AlternativeLoadForecast>(mlContext)

let horizon = 24 * 5
let forecast = forecastingEngine.Predict(horizon = horizon)

In [ ]:
let predChart =
    Seq.zip (testKeys |> Seq.take horizon) forecast.Forecast
    |> fun xy -> Chart.Range(xy,
                             forecast.LowerBound,
                             forecast.UpperBound,
                             mode = StyleParam.Mode.Lines,
                             Color = Colors.toWebColor Colors.Table.Office.blue,
                             RangeColor = Colors.toWebColor Colors.Table.Office.lightBlue)
    |> Chart.withTraceName "Forecast_CI"

let actualChart =
    Seq.zip (testKeys |> Seq.take horizon) (testRows |> Seq.take horizon |> Seq.map (fun r -> r.Target))
    |> fun xy -> Chart.Line(xy,
                            Color = Colors.toWebColor Colors.Table.Office.orange,
                            UseWebGL = true,
                            Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let actualVals = testRows |> Seq.take (24 * 30) |> Seq.map (fun r -> r.Target)
let predVals = forecast.Forecast

let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

<!-- Plotly chart will be drawn inside this DIV -->

0.8457290402462289

In [ ]:
let forecastModel = modelDirectory + "/forecast_model.zip"

mlContext.Model.Save(model, altForecastData.Schema, forecastModel)